# 1. Giới thiệu

Reinforcement Learning (RL) hay Học tăng cường là một nhóm các thuật toán Học máy có thể được huấn luyện bằng cách phản ánh lại xem hành vi gần nhất của chúng trong môi trường nhất định là tốt hay xấu. Trong số đó, các phương pháp khác nhau dựa trên Học giá trị hoặc Học chiến lược đều được đánh giá cao vì có hiệu quả nhất định và có nhiều tiềm năng trong các lĩnh vực khác nhau trong cuộc sống. 

Trong bài tập lớn lần này, em áp dụng một ứng dụng của bài toán Học tăng cường qua môi trường mô phỏng [Lunar Lander - Gymnasium Documentation](https://gymnasium.farama.org/environments/box2d/lunar_lander/). Trong môi trường này, người chơi có trách nhiệm điều khiển tàu vũ trụ hạ cánh an toàn trên bề mặt mặt trăng nơi có những yếu tố bên ngoài tác động như trọng lực, sự nhiễu loạn,....

Xuyên suốt quá trình thực hiện bài tập này, em đã tìm hiểu và áp dụng cũng như huấn luyện mô hình dựa trên một số thuật toán khác nhau, có thể kể đến đầu tiên là Advantage Actor Critic (A2C), một mô hình Học tăng cường lai giữa Học giá trị và Học chiến lược giúp ổn định quá trình huấn luyện bằng cách giảm phương sai qua Actor và Critic. Tuy nhiên, mô hình này không có kết quả tốt trong quá trình huấn luyện nên em đã chuyển sang một mô hình có tính ổn định cao hơn là Proximal Policy Optimization (PPO) kết hợp với Generalized Advantage Estimator (GAE). Trong notebook này, em xin phép trình bày về thuật toán, mô hình, cũng như cách thực thi với môi trường Lunar Lander và báo cáo kết quả thu được.

# 2. Thuật toán

## 2.1. Advantage Actor Critic (A2C)

Trong Học tăng cường, chúng ta muốn tăng xác suất của hành động trong một quỹ đạo tỉ lệ với điểm thường tích luỹ được: $$\nabla_{\theta}J(\theta) = \sum_{t=0}\nabla_{\theta}\log \pi_{\theta}(a_{t}|s_{t})R(\tau)$$
Trong đó $\nabla_{\theta}\log \pi_{\theta}(a_{t}|s_{t})$ là hướng tăng mạnh nhất của log xác suất của việc chọn hành động $a_t$ từ trạng thái $s_t$ , còn $R(\tau)$ là điểm thưởng tích luỹ trên cả quỹ đạo $\tau$.

Nếu điểm tích luỹ cao, chúng ta sẽ tăng xác suất rơi vào cặp trạng thái - hành động đó, và ngược lại.

Điểm thưởng tích luỹ $R(\tau)$ được tính bằng cách lấy mẫu Monte-Carlo: lấy một quỹ đạo và tính điểm thưởng có chiết khấu trên quỹ đạo đó, và dùng giá trị này để tăng / giảm xác suất của mọi hành động trên quỹ đạo này. Nếu điểm thưởng trả về là tốt, vậy mọi hành động trên quỹ đạo đó sẽ được "tăng cường" bằng cách tăng xác suất được lựa chọn. $$R(\tau) = R_{t+1} + \gamma R_{t+2}+\gamma^2R_{t+3}+\dots$$
Tuy nhiên, vì môi trường có những yếu tố ngẫu nhiên trong mỗi lần chơi và tính ngẫu nhiên trong từng chiến lược, các quỹ đạo có thể dẫn tới những điểm thưởng khác nhau, dẫn tới tăng phương sai. Nói cách khác, cùng một trạng thái bắt đầu những có thể dẫn tới nhiều kết quả khác nhau trong những lần chơi khác nhau. 

![](images/image41.png)

**Giải pháp**: Giảm bớt phương sai bằng cách dùng số lượng lớn quỹ đạo, với hi vọng rằng phương sai trong bất kì một quỹ đạo nào cũng sẽ được giảm bớt trong tổng thể và qua đó cho ta một ước lượng thật chuẩn xác về kết quả đúng.

Tuy rằng giải pháp khả thi, nhưng việc dùng nhiều quỹ đạo cùng lúc làm giảm hiệu quả lấy mẫu. Từ đó, thuật toán A2C ra đời nhằm giải quyết vấn đề về phương sai.

Ý tưởng cơ bản: ta có 2 hàm xấp xỉ:
- Actor $\pi_{\theta}(s)$: Thực hiện hành động ngẫu nhiên.
- Critic $V(s)$: Quan sát hành động và phản hồi lại, qua đó Actor sẽ thay đổi chiến lược của mình để chơi tốt hơn.

Hàm Advantage: tính toán xem tại một trạng thái nhất định $s$, việc thực hiện hành động $a$ sẽ tốt hơn như thế nào so với điểm thưởng trung bình của tất cả hành động khác tại trạng thái đó $$A(s,a) = Q(s,a) - V(s)$$
Ta kí hiệu tham số của Actor là $\theta$, tham số của Critic là $\theta_{c}$, learning rate của Actor là $\alpha$, learning rate của Critic là $\alpha_{c}$

Quá trình thực hiện thuật toán như sau:
1. Bắt đầu với trạng thái ban đầu $s_{t}$. Actor lựa chọn một hành động ngẫu nhiên $a_{t}$: $$a_{t} \sim \pi(.|s_{t};\theta)$$
2. Môi trường nhận hành động $a_{t}$, trả về điểm thưởng $r_{t+1}$ cho Critic và trạng thái mới $s_{t+1}$ cho cả Critic và Actor.
3. Critic đánh giá Temporal Difference (TD Error), đây cũng sẽ là giá trị ước lượng thay cho công thức hàm Advantage ở trên: $$\delta_{t} = r_{t+1} + \gamma V(s_{t+1};\theta_{c}) - V(s_{t};\theta_{c})$$
4. Critic và Actor cập nhật trọng số cho mô hình bằng cách dùng TD Error
	1. Cập nhật cho Critic
		1. Hàm loss: $$L_{c}(\theta_{c}) = \frac{1}{2}(r_{t+1} + \gamma V(s_{t+1};\theta_{c}) - V(s_{t};\theta_{c}))^2 = \frac{1}{2}\delta_{t}^2$$
		2. Gradient: $$\nabla_{\theta_{c}}L(\theta_{c}) = \delta_{t}.(\gamma \nabla_{\theta_{c}}V(s_{t+1};\theta_{c}) - \nabla_{\theta_{c}}V(s_{t};\theta_{c}))$$
		3. Gradient Descent: $$\theta_{c} = \theta_{c} - \alpha_{c}.\nabla_{\theta_{c}}L(\theta_{c})$$
	2. Cập nhật cho Actor
		1. Hàm loss: $$L(\theta) = -\log \pi(a_{t}|s_{t};\theta).\delta_{t}$$
		2. Gradient: $$\nabla_{\theta}L(\theta) = -\nabla_{\theta}\log \pi(a_{t}|s_{t};\theta).\delta_{t}$$
		3. Gradient Descent: $$\theta = \theta - \alpha.\nabla_{\theta}L(\theta)$$
5. Đi tới trạng thái tiếp theo $s_t = s_{t+1}$, Actor tiếp tục lựa chọn hành động $a_{t+1}$, sau đó thuật toán lặp lại tới khi kết thúc.

Một số lưu ý về A2C như sau: 
- $\pi(a_{t}|s_{t};\theta)$ là một mạng nơ-ron để chọn hành động dựa trên trạng thái hiện tại, $V(s)$ là một mạng nơ-ron để lấy giá trị điểm thưởng từ trạng thái $s$.
- Với $J(\theta)$ là ước lượng điểm thưởng tích luỹ: $$\nabla_{\theta}J(\theta) \approx E[\nabla_{\theta}\log \pi(a_{t}|s_{t};\theta).\delta_{t}]$$, nói cách khác, $\nabla_{\theta}J(\theta) = -\nabla_{\theta}L(\theta)$, việc cực tiểu hoá hàm loss của Actor đồng nghĩa với việc cực đại hoá ước lượng điểm thưởng tích luỹ.


## 2.2. Proximal Policy Optimization (PPO)

Tuy A2C có thể giải được các bài toán Học tăng cường đơn giản (Như Lunar Lander) trong một số bước hữu hạn, ở một số trường hợp nhất định, thuật toán tỏ ra không hiệu quả, có thể rơi vào trường hợp thay đổi chiến lược quá nhanh quá mạnh hoặc không thật sự học được gì (qua thực tế huấn luyện). Chính vì thế, em đã thay đổi sang một thuật toán mang tính ổn định cao hơn là Proximal Policy Optimization (PPO), tích hợp thêm Generalised Advantage Estimation (GAE) để ước lượng Advantage. 

Điểm nổi bật của thuật toán PPO so với A2C là giữ được cấu trúc cũ (Actor, Critic và Advantage), đồng thời bảo đảm việc cập nhật chiến lược sẽ không quá khác biệt, điều này mang lại 2 tác dụng:
- Cập nhật chiến lược nhỏ giúp việc huấn luyện dễ hội tụ về một giải pháp tối ưu
- Cập nhật chiến lược quá lớn có thể khiến chiến lược mới rất xấu, mất nhiều thời gian để khôi phục hoặc thậm chí không thể khôi phục được

**Lưu ý**: Trước khi đi sâu hơn vào [bài báo sau đây](https://arxiv.org/pdf/1707.06347), để không nhầm lẫn giữa cách kí hiệu trong báo cáo này và trong bài báo, quy định rằng $J(\theta)$ là hàm mục tiêu cần cực đại hoá và tương tự $L(\theta)$ là hàm loss cần cực tiểu hoá (Giống kí hiệu ở phần A2C).

<a id="policy-objective-function"></a>
### 2.2.1. Policy Objective Function

Công thức: $$J^{PG}(\theta) = \hat{E}_{t}[\log \pi_{\theta}(a_{t}|s_{t}).\hat{A}_{t}]$$
Trong đó:
- $\log \pi_\theta(a_{t}|s_{t})$ là log xác suất lựa chọn hành động $a_t$ tại trạng thái $s_t$.
- $\hat{A}_{t}$ là ước lượng giá trị Advantage của $a_t$ tại thời điểm $t$ bằng Monte-Carlo (tính toán cho cả một quỹ đạo). Cụ thể hơn thì $\hat{A}_{t} = G_{t} - V(s_{t})$ trong đó:
	- $G_{t}$ là tổng điểm thưởng có chiết khấu từ thời điểm $t$: $$G_{t} = R_{t+1} + \gamma R_{t+2} + \dots = \sum_{k = 0}^{\infty}\gamma^kR_{t+k+1}$$
	- $V(s)$ là ước lượng điểm thưởng có chiết khấu bắt đầu từ trạng thái $s$
- Nếu $\hat{A}_{t}$ dương, điều đó có nghĩa là đạo hàm dương, tức là ta tăng xác suất thực hiện hành động đó, và tương tự với chiều ngược lại.

Một vấn đề xảy ra, mặc dù ta muốn mô hình thực hiện hành động có điểm thưởng cao hơn và tránh những hành động có hại, nhưng trong quá trình huấn luyện:
- Nếu cập nhật chiến lược nhỏ, quá trình học sẽ rất chậm.
- Nếu cập nhật chiến lược lớn, sẽ có rất nhiều phương sai, biến thiên trong quá trình học

Chính vì những lí do kể trên, ta sẽ đi tới công thức tiếp theo, giúp đảm bảo sự thay đổi trong chiến lược sẽ diễn ra từ từ và được kiểm soát trong một khoảng nhất định.

### 2.2.2. Clipped Surrogate Objective Function

Trước tiên, ta có khái niệm về hàm tỉ lệ: $$r_{t}(\theta) = \frac{\pi_{\theta}(a_{t}|s_{t})}{\pi_{\theta_{old}}(a_{t}|s_{t})}$$ 
Tỉ lệ này được tính bằng cách lấy xác suất lựa chọn hành động $a_{t}$ tại trạng thái $s_t$ trong chiến lược hiện tại, chia cho xác suất tương tự nhưng trong chiến lược trước đó. Ta có thể thấy có 2 khả năng xảy ra:
- Nếu $r_{t}(\theta) > 1$, có nhiều khả năng xảy ra hành động $a_{t}$ hơn trong chiến lược hiện tại so với chiến lược trước đó
- Nếu $r_t(\theta) < 1$, có ít khả năng xảy ra hành động $a_{t}$ hơn trong chiến lược hiện tại so với chiến lược trước đó
Hàm tỉ lệ này là một cách đơn giản để ước tính sự chệch hướng giữa chiến lược cũ và mới.

Tiếp đến, ta có công thức cho Clipped Surrogate Objective Function: $$J^{CLIP}(\theta) = \mathbb{\hat{E}}_{t}[\min(r_{t}(\theta)\hat{A}_{t},clip(r_{t}(\theta), 1-\epsilon, 1+\epsilon)\hat{A}_{t})]$$
Trong công thức này, hàm tỉ lệ ở trên được dùng thay thế log xác suất trong hàm **Policy Objective Function** ở trên. Ta xem xét công thức này qua 2 phần trong hàm $\min$

### 2.2.3. The unclipped part (Phần bên trái - không bị cắt)

Công thức: $$J^{CPI}(\theta) = \mathbb{\hat{E}}_{t}\left[\frac{\pi_{\theta}(a_{t}|s_{t})}{\pi_{\theta_{old}}(a_{t}|s_{t})}\hat{A}_{t}\right] = \mathbb{\hat{E}}_{t}\left[r_{t}(\theta)\hat{A}_{t}\right]$$
Tuy nhiên, nếu không có ràng buộc, việc cực đại hoá hàm này sẽ dẫn tới cập nhật chiến lược quá lớn (điều mà ta không muốn), vậy nên ta phải điều chỉnh hàm mục tiêu này để trừng phạt những thay đổi mà khiến cho $r_{t}(\theta)$ cách 1 quá xa.

### 2.2.4. The clipped part (Phần bên phải - bị cắt)

Công thức: $$clip(r_{t}(\theta), 1 - \epsilon, 1 + \epsilon)\hat{A}_{t}$$
Ở công thức này, ta đảm bảo sự thay đổi chiến lược sẽ không đi quá xa, chỉ được nằm trong khoảng $[1−ϵ,1+ϵ]$ . Theo các thí nghiệm trong [bài báo](https://arxiv.org/pdf/1707.06347), với $\epsilon = 0.2$ ta có điểm chuẩn hoá cao nhất. 

![](images/image42.png)

**Kết luận**: ta lấy $\min$ của phần không bị cắt và phần bị cắt, vì vậy hàm mục tiêu cuối cùng sẽ là cận dưới của phần không bị cắt. Bằng cách này, nếu hàm tỉ lệ giúp hàm mục tiêu cải thiện tốt hơn, ta sẽ kệ nó, nhưng nếu nó làm hàm mục tiêu tệ hơn, ta sẽ áp dụng $\min$ để đảm bảo nó không đi quá xa.

### 2.2.5. Trực quan hoá công thức Clipped Surrogate Objective Function

![](images/image43.png)

Ta quan sát được tổng cộng 6 trường hợp tương ứng (Trong hình, $p_{t}(\theta)$ là $r_{t}(\theta)$). Xét tại thời điểm $t$:

- Trường hợp 1 và 2: Hàm tỉ lệ đã nằm sẵn trong khoảng $[1−\epsilon,1 + \epsilon]$
	- Trường hợp 1: Hàm advantage dương $\to$ Hành động hiện tại tốt hơn trung bình các hành động khác $\to$ khuyến khích chiến lược hiện tại tăng xác suất chọn hành động hiện tại .
	- Trường hợp 2: Hàm advantage âm $\to$ Hành động hiện tại tệ hơn trung bình các hành động khác $\to$ KHÔNG khuyến khích chiến lược hiện tại tăng xác suất chọn hành động hiện tại.
- Trường hợp 3 và 4: Hàm tỉ lệ nằm dưới khoảng ($r_{t}(\theta) < 1 - \epsilon$) $\to$ Xác suất chọn hành động hiện tại với chiến lược hiện tại ít hơn nhiều so với chiến lược cũ.
	- Trường hợp 3: Hàm advantage dương $\to$ Tăng xác suất chọn hành động hiện tại.
	- Trường hợp 4: Hàm advantage âm $\to$ Xác suất đã thấp sẵn, ta càng không muốn giảm xác suất đó đi, vì thế ta không cập nhật hệ số.
- Trường hợp 5 và 6: Hàm tỉ lệ nằm trên khoảng ($r_{t}(\theta) > 1 + \epsilon$) $\to$ Xác suất chọn hành động hiện tại với chiến lược hiện tại cao hơn nhiều so với chiến lược cũ.
	- Trường hợp 5: Hàm advantage dương $\to$ Xác suất đã cao sẵn, ta càng không muốn tăng xác suất đó lên, vì thế ta không cập nhật hệ số.
	- Trường hợp 6: Hàm advantage âm $\to$ Giảm xác suất chọn hành động hiện tại.

**Kết luận**: 
- Ta chỉ cập nhật chiến lược đối với phần không bị cắt. Cụ thể hơn:
	- Ta cập nhật chiến lược nếu hàm tỉ lệ nằm trong khoảng $[1−\epsilon,1 + \epsilon]$.
	- Ta cập nhật chiến lược nếu hàm tỉ lệ nằm ngoài khoảng $[1−\epsilon,1 + \epsilon]$, nhưng hàm advantage giúp ta tiến gần hơn khoảng này.
- Ta giới hạn khoảng cách mà chiến lược mới có thể thay đổi so với chiến lược cũ.

### 2.2.6. Final PPO's Actor Critic Objective Function

Công thức cuối cùng ta có: $$J^{CLIP + VF + S}_{t}(\theta) = \mathbb{\hat{E}}_{t}\left[J^{CLIP}_{t}(\theta) - c_{1}L^{VF}_{t}(\theta) + c_{2}S[\pi_{\theta}](s_{t})\right] $$
Trong đó:
- $\theta$ là tham số mô hình.
- $c_1, c_2$ là hệ số.
- $J^{CLIP}_{t}(\theta)$ là **hàm mục tiêu Clipped Surrogate Objective Function** (phần 2.2.2.)
- $L^{VF}_{t}(\theta)$ là hàm mất mát với sai số bình phương ($L^{VF}_{t}(\theta) = (V_{\theta}(s_{t}) -V^{target})^2$).
- $S[\pi_{\theta}](s_{t})$ là entropy để đảm bảo mô hình có thể đi khám phá.

Nhưng đây là hàm mục tiêu, vậy hàm mất mát là gì? Và ta sẽ tối ưu nó ra sao?

Rất đơn giản, chỉ cần để dấu trừ đằng trước (tức là lật tất cả các dấu bên trong), ta sẽ có hàm mất mát để có thể cực tiểu hoá. $$L^{CLIP + VF + S}_{t}(\theta) = \mathbb{\hat{E}}_{t}\left[-J^{CLIP}_{t}(\theta) + c_{1}L^{VF}_{t}(\theta) - c_{2}S[\pi_{\theta}](s_{t})\right] $$

## 2.3. Generalised Advantage Estimation (GAE)

Thay vì công thức tính Advantage $\hat{A}_{t}$ thông thường, ta dùng một công thức để ước lượng giá trị của nó trong [bài báo dưới đây](https://arxiv.org/pdf/1506.02438). 

Với công thức tính Temporal Difference Error (TD Error) ở trên và ta biết rằng TD Error có thể dùng để ước lượng $\hat{A}_{t}$, ta có công thức sau:

- Công thức ước lượng $\hat{A}_{t}$ cho $k$ bước: 

![](images/image44.png)
- Công thức khi $k \to \infty$: 

![](images/image45.png)
- Công thức GAE: Công thức này được định nghĩa là trung bình ước lượng của $\hat{A}_{t}$ trong $k$ bước có trọng số luỹ thừa: 

![](images/image46.png)
- Khi ta thay đổi siêu tham số $\lambda = 0$ và $\lambda = 1$, ta có: 
	- Với $\lambda = 0$, phương trình chính là TD Error dùng để ước lượng $\hat{A}_{t}$ cho 1 bước.
	- Với $\lambda = 1$, phương trình chính là TD Error Monte-Carlo (tính toán cho cả 1 quỹ đạo).

![](images/image47.png)

# 3. Áp dụng thuật toán vào code

Code được trình bày theo thuật toán trong [bài báo này](https://fse.studenttheses.ub.rug.nl/25709/1/mAI_2021_BickD.pdf).

![](images/image48.png)

Một số thay đổi:
- Để đơn giản hoá, chỉ dùng 1 agent để thu thập training data ($N = 1$).
- Thay vì tính $V^{target}_{t}$ trước sau đó mới tính $A_{t}$ thì trong code, nhờ áp dụng thuật toán GAE
nên ta sẽ tính $\hat{A_{t}}$ trước, sau đó mới tính $V^{target}_{t}$ bằng cách cộng thêm với $V_{w}(s_{t})$ (thông qua Critic Network để lấy value của $s_t$).
- Thay vì xét với từng $example$ $e \in M$, ta sẽ tính toán và tối ưu hoá trực tiếp với minibatch $M$.

## 3.1. Code A2C

# Mô tả quá trình

Thuật toán A2C em áp dụng để huấn luyện mô hình cho kết quả rất không tốt, mô hình hầu như không học được và cho rewards âm. Vì phần code của thuật toán này và thuật toán PPO khá giống nhau nên em xin trình bày chi tiết và đầy đủ hơn ở phần sau. Phiên bản này cũng là một phiên bản chưa chỉn chu, hoàn thiện, chưa có docstrings.

In [ ]:
# Libraries and dependencies
import gymnasium as gym
import numpy as np
from tqdm import tqdm
from gymnasium.wrappers import RecordVideo
import torch
import torch.nn as nn
import torch.distributions as distributions
import torch.optim as optim
import matplotlib.pyplot as plt
# Training environment (wind disabled)
env = gym.make(
    "LunarLander-v3",
    continuous=False,     
    gravity=-10.0,        
    enable_wind=False,   
    wind_power=15.0,      
    turbulence_power=1.0, 
    render_mode="rgb_array" 
)

In [ ]:
# Evaluation environment (wind enabled)
video_env = gym.make(
    "LunarLander-v3",
    continuous=False,     
    gravity=-10.0,        
    enable_wind=False,   
    wind_power=15.0,      
    turbulence_power=1.0, 
    render_mode="rgb_array" 
)

# Record video every 100 epochs
video_env = RecordVideo(video_env, video_folder="a2c_1e6_64_2048_3e4_gae", episode_trigger=lambda x: x % 1000 == 0)   
space_dim = env.observation_space.shape[0]      # Observation space: 8-dimensional vector
action_dim = env.action_space.n                 # Action space: 4 discrete actions


In [ ]:
# Training parameters (following paper)
gamma = 0.99                # Discount factor
lr = 3e-4                   # Learning rate
lamb = 0.95                 # Generalised Advantage Estimation (GAE) lambda
epsilon = 0.2               # Clipping value
h = 0.01                    # Entropy coefficient
max_timesteps = 1e6         # Maximal number of iterations
eval_episodes = 100         # Episodes for evaluation
N = 1                       # Number of agents collecting training data
T = 2048                    # Maximal trajectory length
K = 10                      # Number of epoches per update
minibatch_size = 64         # Size of a mini batch
number_minibatches = N * T / minibatch_size     # Number of mini batches
actor_losses = []           # For plotting
critic_losses = []
eval_rewards = []            
index = []

In [ ]:
# The network to select an action
ActorNetwork = nn.Sequential(
    nn.Linear(space_dim, 128),
    nn.LeakyReLU(),
    nn.Linear(128, 128),
    nn.LeakyReLU(),
    nn.Linear(128, action_dim)
)

# The network to get value of a state
CriticNetwork = nn.Sequential(
    nn.Linear(space_dim, 128),
    nn.LeakyReLU(),
    nn.Linear(128, 128),
    nn.LeakyReLU(),
    nn.Linear(128, 1)
)

# Optimizer using Adam Gradient Descent
actor_optimizer = optim.Adam(ActorNetwork.parameters(), lr=lr)
critic_optimizer = optim.Adam(CriticNetwork.parameters(), lr=lr)


In [ ]:
def select_action(state_tensor):
    value = CriticNetwork(state_tensor)
    action_pred = ActorNetwork(state_tensor)                    
    dist = distributions.Categorical(logits=action_pred)
    action = dist.sample()
    log_prob = dist.log_prob(action)

    return action, value, log_prob
def collect_training_data(state, action, reward, log_prob, value, done, states, actions, rewards, log_probs, values, dones):
    states.append(state)         # Collect states
    actions.append(action)        # Collect actions from Actor Network
    rewards.append(reward)        # Collect rewards
    log_probs.append(log_prob)      # Collect lob_probs
    values.append(value)         # Collect values from Critic Network
    dones.append(done)          # Collect done (0 or 1)
def compute_GAE(next_value, rewards, values, dones):
    advantages = []
    GAE = 0
    for t in reversed(range(len(rewards))):
        delta = rewards[t] + gamma * next_value * (1 - dones[t]) - values[t]      # TD error
        GAE = delta + gamma * lamb * (1 - dones[t]) * GAE
        advantages.insert(0, GAE)
        next_value = values[t].item()
    return advantages
def compute_advantages(next_state_tensor: torch.Tensor, rewards: list, values: list, dones: list):
    next_value = CriticNetwork(next_state_tensor)                                           # Calculate the next value
    advantages = compute_GAE(next_value, rewards=rewards, values=values, dones=dones)       # Calculate advantage using GAE
    advantages = torch.FloatTensor(advantages)              
    return advantages
def get_parameterized_policy(batch_states, batch_actions):
    optimized_value = CriticNetwork(batch_states)
    optimized_action_preds = ActorNetwork(batch_states)
    optimized_dist = distributions.Categorical(logits=optimized_action_preds)
    optimized_log_probs = optimized_dist.log_prob(batch_actions)

    return optimized_value, optimized_dist, optimized_log_probs
def evaluation(eval_episodes):
    total_reward = 0
    for _ in range(eval_episodes):
        episode_reward = 0
        state, _ = video_env.reset()
        done = False
        while not done:
            state_tensor = torch.FloatTensor(state)
            action_pred = ActorNetwork(state_tensor)
            action = torch.argmax(action_pred).item()
            state, reward, terminated, truncated, _ = video_env.step(action)
            episode_reward += reward
            done = terminated or truncated

        total_reward += episode_reward
    return total_reward / eval_episodes  

def moving_average(data, window_size=10):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

In [ ]:
state, _ = env.reset()      # Initialize state s_t
timesteps = 0

while timesteps < max_timesteps:
    states, actions, rewards, log_probs, values, dones = [], [], [], [], [], []
    print(timesteps)

    for _ in range(T):              # Collect T timesteps for 1 rollout
        action, value, log_prob = select_action(torch.FloatTensor(state)) # Get action a_t, value V(s)_t, and old policy given action a_t

        next_state, reward, terminated, truncated, _ = env.step(action.item())      # Advance simulation one time step
        done = terminated or truncated

        # Collect training data
        collect_training_data(state, action, reward, log_prob, value, done, states, actions, rewards, log_probs, values, dones)  

        state = next_state      # Move to next state
        timesteps += 1          # Increase timesteps

        if done:
            state, _ = env.reset()      # If an episode is done, reset the state
    
    # Compute advantages
    advantages = compute_advantages(next_state_tensor=torch.FloatTensor(state), rewards=rewards, values=values, dones=dones)     # Calculate advantages
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)               # Normalize advantages
    returns = advantages + torch.FloatTensor(values)        # Calculate V_target_t = A_t + V_w(s_t)

    # Convert data for training 
    states = torch.FloatTensor(states)
    actions = torch.LongTensor(actions)
    old_log_probs = torch.FloatTensor(log_probs)
    returns = returns.detach()

    # Optimizing the surrogate loss
    for k in range(K):          
        for i in torch.randperm(len(states)).split(minibatch_size):        # Sample over batches with size 'minibatch_size'      
            batch_states = states[i]
            batch_actions = actions[i]
            batch_old_log_probs = old_log_probs[i]
            batch_returns = returns[i]
            batch_advantages = advantages[i]

            optimized_value, optimized_dist, optimized_log_probs = get_parameterized_policy(batch_states=batch_states, batch_actions=batch_actions)
            H_entropy = optimized_dist.entropy().mean()

            actor_loss = -(optimized_log_probs * batch_advantages).mean() - h * H_entropy
            critic_loss = (optimized_value.squeeze() - batch_returns).pow(2).mean()  

            actor_losses.append(actor_loss.item())
            critic_losses.append(critic_loss.item())
            
            critic_optimizer.zero_grad()
            critic_loss.backward()
            critic_optimizer.step()

            actor_optimizer.zero_grad()
            actor_loss.backward()
            actor_optimizer.step()

    if timesteps % T == 0:
        try:
            avg_rewards = evaluation(eval_episodes)
            print(f"In current timestep: {timesteps}, average reward: {avg_rewards}", flush=True)
            index.append(timesteps)
            eval_rewards.append(avg_rewards)
        except Exception as e:
            print(f"Error in evaluation: {e}", flush=True)

In [ ]:
plt.plot(moving_average(actor_losses))
plt.title("Actor Loss")
plt.xlabel("Rollout")
plt.ylabel("Loss")
plt.show()

![](images/image49.png)

In [ ]:
plt.plot(moving_average(critic_losses))
plt.title("Critic Loss")
plt.xlabel("Rollout")
plt.ylabel("Loss")
plt.show()

![](images/image50.png)

In [ ]:
plt.plot(index, eval_rewards)
plt.title("Evaluation Rewards")
plt.xlabel("Timesteps")
plt.ylabel("Rewards")
plt.show()

![](images/image51.png)

## 3.2 Code PPO

### Mô tả quá trình

Đây là phiên bản code PPO đã được tối ưu và hoàn thiện nhất có thể. Trong phần code này, em thực hiện training theo thuật toán ở phía trên. Trong quá trình huấn luyện, em lần lượt thay đổi với từng yếu tố khác nhau: 
- T = 2048 hoặc 4096 (số lượng timesteps trong một rollout).
- minibatch_size = 64 hoặc 256 (kích cỡ của một minibatch).
- Có gió hoặc không có gió.

Nhờ thay đổi những siêu tham số này, em tổng hợp kết quả của 8 lần training và thời gian training của chúng. Kết quả training được đánh giá qua 5 tiêu chí khác nhau:
- Loss của Actor Network
- Loss của Critic Network
- Điểm trung bình qua thời gian
- Độ lệch chuẩn qua thời gian
- Tỉ lệ hạ cánh thành công qua thời gian

### Quá trình training sẽ diễn ra tuần tự như sau:

1. Reset lại môi trường, khởi tạo state đầu tiên.
2. Cho timestep chạy từ 0 đến max_timesteps.
3. Với mỗi một lần rollout tương ứng với ***T*** timesteps:
    1. Dùng hàm **select_action** để chọn một ***action*** ngẫu nhiên theo chiến lược của Actor Network, trả về ***value*** tại state hiện tại theo Critic Network, và ***log-probability*** của action đó.
    2. Từ action này, lấy ra ***next_state*** tiếp theo của môi trường, điểm thưởng cho action đó, cũng như trạng thái của môi trường (đã ***terminated*** hoặc ***truncated*** chưa).
    3. Dữ liệu huấn luyện (***state, action, reward, log_prob, value, done***) sẽ được lưu vào danh sách để phục vụ huấn luyện sau này.
    4. Nếu môi trường kết thúc (***done***), reset lại môi trường và lấy ***state*** mới.
    5. Tăng số lượng ***timesteps*** để theo dõi quá trình huấn luyện đã diễn ra bao lâu.
4. Sau mỗi lượt rollout, ta sẽ tính Advantage và Returns
    1. Tính ***advantages*** bằng Generalised Advantage Estimate (GAE) từ ***rewards*** và ***values***.
    2. Chuẩn hoá ***advantages*** để giúp quá trình training được ổn định.
    3. Tính ***returns*** = ***advantages*** + ***values*** (Đây cũng chính là $V_{target}$ cho Critic).
5. Chuyển đổi dữ liệu huấn luyện thu thập được qua Tensor để dễ dàng làm việc với mạng nơ-ron.
6. Tối ưu hoá chính sách: 
    1. Lặp với ***K*** epochs.
    2. Chia dữ liệu thành các minibatch ngẫu nhiên với kích cỡ là ***minibatch_size***.
    3. Với mỗi minibatch:
        1. Từ mô hình mạng nơ-ron hiện tại, tính ***optimized_value***, ***optimized_dist***, ***optimized_log_probs***.
        2. Tính ***L_clip*** là PPO Loss theo clipping surrogate function, ***L_v*** là Critic loss và ***H_entropy*** là entropy của chiến lược để khuyến khích khám phá (exploration thay vì exploitation).
        3. Tính ***actor_loss = L_clip - h * H_entropy*** trong đó ***h*** là hệ số entropy.
        4. Tính ***critic_loss = v * L_v*** trong đó ***v*** là hệ số của Critic loss.
        5. Cập nhật ***Actor Network*** và ***Critic Network*** bằng backpropagation.
        6. Lưu lại giá trị loss để plot.
7. Sau mỗi lượt rollout, đánh giá mô hình tại thời điểm đó.
    1. Gọi hàm ***evaluation_and_save()*** để đánh giá các kết quả sau qua 100 lượt chơi thử:
        1. Điểm thưởng trung bình (***avg_rewards***)
        2. Độ lệch chuẩn (***std_rewards***)
        3. Tỉ lệ hạ cánh thành công (***success_rate***)
    2. In kết quả đánh giá để kiểm tra trong quá trình training.
    3. Lưu những kết quả này để plot.
 

In [ ]:
# Libraries and dependencies
import gymnasium as gym
import numpy as np
from gymnasium.wrappers import RecordVideo
import torch
import torch.nn as nn
import torch.distributions as distributions
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# Training parameters (following paper)
gamma = 0.99                # Discount factor
lr = 3e-4                   # Learning rate
lamb = 0.95                 # Generalised Advantage Estimation (GAE) lambda
epsilon = 0.2               # Clipping value
h = 0.01                    # Entropy coefficient
v = 0.5                     # Value loss coefficient
max_timesteps = 1e6         # Maximal number of iterations
eval_episodes = 100         # Episodes for evaluation
N = 1                       # Number of agents collecting training data
T = 4096                    # Maximal trajectory length
K = 10                      # Number of epoches per update
minibatch_size = 64         # Size of a mini batch
number_minibatches = N * T / minibatch_size     # Number of mini batches
max_reward = 0
actor_losses = []          
critic_losses = []
eval_rewards = []
eval_standard_deviations = []
success_rates = []            
index = []

In [ ]:
# Training and Evaluating environment (wind is enabled/disabled)
env = gym.make(
    "LunarLander-v3",
    continuous=False,     
    gravity=-10.0,        
    enable_wind=False,   
    wind_power=15.0,      
    turbulence_power=1.0, 
    render_mode="rgb_array" 
)

video_folder = f"ppo_wind_{max_timesteps}_{minibatch_size}_{T}_{lr}" if env.unwrapped.enable_wind else f"ppo_{max_timesteps}_{minibatch_size}_{T}_{lr}"

# Record video every 1000 epochs
video_env = RecordVideo(env, video_folder=f"{video_folder}", episode_trigger=lambda x: x % 1000 == 0) 

# Video for the best model
best_env = RecordVideo(env, video_folder=f"best_of_{video_folder}")

space_dim = env.observation_space.shape[0]                  # Observation space: 8-dimensional vector
action_dim = env.action_space.n                             # Action space: 4 discrete actions

In [ ]:
# The network to select an action
ActorNetwork = nn.Sequential(
    nn.Linear(space_dim, 128),
    nn.LeakyReLU(),
    nn.Linear(128, 128),
    nn.LeakyReLU(),
    nn.Linear(128, action_dim)
)

# The network to get value of a state
CriticNetwork = nn.Sequential(
    nn.Linear(space_dim, 128),
    nn.LeakyReLU(),
    nn.Linear(128, 128),
    nn.LeakyReLU(),
    nn.Linear(128, 1)
)

# Optimizer using Adam Gradient Descent
actor_optimizer = optim.Adam(ActorNetwork.parameters(), lr=lr)
critic_optimizer = optim.Adam(CriticNetwork.parameters(), lr=lr)

In [ ]:
def select_action(state_tensor: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Select a random action following a policy.

    Parameters
    ----------
    state_tensor : torch.FloatTensor
        An 8-dimensional vector of observation space.

    Returns
    ----------
    action : torch.Tensor
        The discrete action taken 0, 1, 2, 3.
    value : torch.Tensor 
        Value estimated at the state from the CriticNetwork.
    log_prob : torch.Tensor: 
        Log-probability of the chosen action.
    """
    value = CriticNetwork(state_tensor)
    action_pred = ActorNetwork(state_tensor)    # Tensor of raw logits              
    dist = distributions.Categorical(logits=action_pred)    # Form a categorical distribution from the logits (perform softmax internally)
    action = dist.sample()              # Sample an action from the distribution
    log_prob = dist.log_prob(action)

    return action, value, log_prob

In [ ]:
def collect_training_data(state, action, reward, log_prob, value, done, states, actions, rewards, log_probs, values, dones):
    """
    Collect training data during a rollout.

    Parameters
    ----------
    state : numpy.ndarray
        The environment state at a timestep.
    action : torch.Tensor
        The action taken at the given state.
    reward : numpy.float64
        Reward received for taking the action.
    log_prob : torch.Tensor
        Log-probability of the action.
    value : torch.Tensor
        Value estimated for the state.
    done : bool
        Whether the episode has ended at this timestep.
    states : list of numpy.ndarray
        List to store collected states.
    actions : list of torch.Tensor
        List to store collected actions.
    rewards : list of numpy.float64
        List to store collected rewards.
    log_probs : list of torch.Tensor
        List to store collected log-probabilities.
    values : list of torch.Tensor
        List to store collected state-value estimates.
    dones : list of bool
        List to store episode termination flags.

    Returns
    -------
    None  
    """
    states.append(state)         # Collect states
    actions.append(action)        # Collect actions from Actor Network
    rewards.append(reward)        # Collect rewards
    log_probs.append(log_prob)      # Collect lob_probs
    values.append(value)         # Collect values from Critic Network
    dones.append(done)          # Collect done (0 or 1)

In [ ]:
def compute_GAE(next_value, rewards, values, dones) -> list:
    """
    Compute the Generalized Advantage Estimate (GAE).

    Parameters
    ----------
    next_value : numpy.ndarray
        Value estimate of the next state after the last timestep.
    rewards : list of numpy.float64
        Collected rewards at each timestep.
    values : list of torch.Tensor
        Collected state-value estimates at each timestep.
    dones : list of bool
        Episode termination flags (True if episode ended at that timestep).

    Returns
    -------
    advantages : list of numpy.float64
        Advantage estimates, one per timestep.   
    """
    advantages = []
    GAE = 0
    for t in reversed(range(len(rewards))):
        delta = rewards[t] + gamma * next_value * (1 - dones[t]) - values[t]
        GAE = delta + gamma * lamb * (1 - dones[t]) * GAE
        advantages.insert(0, GAE)
        next_value = values[t].item()
    return advantages


In [ ]:
def compute_advantages(next_state_tensor: torch.Tensor, rewards: list, values: list, dones: list) -> list[torch.Tensor]:
    """
    Get Generalised Advantage Estimate and convert to Tensor.

    Parameters
    ----------
    next_state_tensor : torch.Tensor
        An 8-dimensional vector of observation space indicating the next state.
    rewards : list of numpy.float64
        List to store collected rewards.
    values : list of torch.Tensor
        List to store collected value estimates.
    dones : list of bool
        List to store episode termination indicators.

    Returns
    -------
    advantages : list of numpy.float64
        List of advantage estimates, one for each timestep.   
    """
    next_value = CriticNetwork(next_state_tensor)                                           # Calculate the next value
    advantages = compute_GAE(next_value, rewards=rewards, values=values, dones=dones)       # Calculate advantage using GAE
    advantages = torch.FloatTensor(advantages)              
    return advantages

In [ ]:
def get_parameterized_policy(batch_states, batch_actions) -> tuple[torch.Tensor, torch.distributions.Categorical, torch.Tensor]:
    """
    Under the current policy, calculate value estimate of a batch of states, 
    distribution of a batch of actions, and their Log-probability.

    Parameters
    ----------
    batch_states : torch.FloatTensor
        Batch of states, where each state is an 8-dimensional vector.
    batch_actions : torch.FloatTensor
        Batch of actions taken corresponding to the batch states.

    Returns
    -------
    optimized_value : torch.Tensor
        Predicted state values from the Critic Network.
    optimized_dist : torch.distributions.Categorical
        Action distribution over discrete actions from the Actor Network.
    optimized_log_probs : torch.Tensor
        Log-probabilities of the batch actions under the current policy. 
    """
    optimized_value = CriticNetwork(batch_states)
    optimized_action_preds = ActorNetwork(batch_states)
    optimized_dist = distributions.Categorical(logits=optimized_action_preds)
    optimized_log_probs = optimized_dist.log_prob(batch_actions)

    return optimized_value, optimized_dist, optimized_log_probs

In [ ]:
def calculate_clipped_surrogate_loss(batch_old_log_probs, batch_returns, batch_advantages, optimized_value, optimized_dist, optimized_log_probs):
    """
    Computes the PPO clipped surrogate objective, value function loss, and entropy bonus.
    
    Parameters
    ----------
    batch_old_log_probs : torch.Tensor
        Batch of Log-probabilities of actions under the old policy.
    batch_returns : torch.Tensor
        Batch of all estimated returns (reward to go) for each state.
    batch_advantages : torch.Tensor
        Batch of GAE for each action.
    optimized_value : torch.Tensor
        Value estimate from the current Critic Network.
    optimized_dist : torch.distributions.Categorical
        Action distribution from the current Actor Network.
    optimized_log_probs : torch.Tensor
        Batch of Log-probabilities of actions under the current policy.

    Returns
    -------
    L_clip : torch.Tensor
        Clipped surrogate loss for the policy.
    L_v : torch.Tensor
        Value function loss using Mean Squared Error.
    H_entropy : torch.Tensor
        Mean entropy bonus to encourage exploration.  
    """
    ratio = (optimized_log_probs - batch_old_log_probs).exp()                                   # Ratio = divergence between old and current policy
    unclipped_objective = ratio * batch_advantages                                              # The unclipped part
    clipped_objective = torch.clamp(ratio, 1 - epsilon, 1 + epsilon) * batch_advantages         # The clipped part: Clip the ratio to range [1 - ε, 1 + ε]

    L_clip = -torch.min(unclipped_objective, clipped_objective).mean()          # Take the smaller part
    L_v = (optimized_value.squeeze() - batch_returns).pow(2).mean()             # Squared-error value loss
    H_entropy = optimized_dist.entropy().mean()                                 # Entropy bonus: Ensure sufficient exploration

    return L_clip, L_v, H_entropy

In [ ]:
def evaluation_and_save(eval_episodes, max_reward):
    """
    Utility function to evaluate average rewards over 100 episodes, and save the model with the best reward recorded.

    Parameters
    ----------
    eval_episodes : int
        Number of episodes to evaluate.
    max_reward : int
        Maximum reward ever recorded.

    Returns
    -------
    avg_reward : float
        Average reward over the evaluated episodes.
    std_reward : float
        Standard deviation of the rewards.
    success_rate : float
        Percentage of episodes with a reward of at least 200.
    """
    success_rate = 0
    reward_store = []

    with torch.no_grad():
        for _ in range(eval_episodes):
            episode_reward = 0
            state, _ = video_env.reset()
            done = False
            while not done:
                state_tensor = torch.FloatTensor(state)
                action_pred = ActorNetwork(state_tensor)
                action = torch.argmax(action_pred).item()
                state, reward, terminated, truncated, _ = video_env.step(action)
                episode_reward += reward
                done = terminated or truncated

            reward_store.append(episode_reward)

            if episode_reward >= 200:
                success_rate += 1

            if episode_reward > max_reward:
                max_reward = episode_reward
                torch.save({'Actor': ActorNetwork.state_dict(), 'Critic': CriticNetwork.state_dict()}, "best_model_now.pth")
    return sum(reward_store)/eval_episodes, np.std(reward_store), success_rate / eval_episodes * 100

In [ ]:
def show_best_results():
    """
    Show the mp4 video of the model with the best result, over the whole training process.
    
    Parameters
    ----------
    None

    Returns
    -------
    None
    """
    state_dict = torch.load("best_model_now.pth")

    ActorNetwork.load_state_dict(state_dict=state_dict['Actor'])
    CriticNetwork.load_state_dict(state_dict=state_dict['Critic'])

    best_reward = 0

    with torch.no_grad():
        state, _ = best_env.reset()
        done = False
        while not done:
            state_tensor = torch.FloatTensor(state)
            action_pred = ActorNetwork(state_tensor)
            action = torch.argmax(action_pred).item()
            state, reward, terminated, truncated, _ = best_env.step(action)
            best_reward += reward
            done = terminated or truncated

    print(f"Best reward collected: {best_reward}")

    best_env.close()

In [ ]:
def train(env, max_timesteps, K, T, minibatch_size, max_reward, actor_losses, critic_losses, index, eval_rewards, eval_standard_deviations, success_rates):
    """Main function to train the model.

    Parameters
    ----------
    env : gymnasium.Env
        The Lunar Lander V3 OpenAI Gymnasium environment.
    max_timesteps : int
        Maximum timesteps to train.
    K : int
        Number of epochs to optimize model.
    T : int
        Number of timesteps for a rollout.
    minibatch_size : int
        Size of a mini batch.
    max_reward : int
        Maximum reward ever recorded.
    actor_losses : list
        List to collect loss from Actor Network.
    critic_losses : list
        List to collect loss from Critic Network.
    index : list
        List to collect indices for plotting.
    eval_rewards : list
        List to collect average rewards over the training process.
    eval_standard_deviations : list
        List to collect standard deviations over the training process.
    success_rates : list
        List to collect success landing ratio over the training process.

    Returns
    -------
    None
    """
    
    state, _ = env.reset()      # Initialize state s_t
    timesteps = 0

    while timesteps < max_timesteps:
        states, actions, rewards, log_probs, values, dones = [], [], [], [], [], []
        print(timesteps)

        for _ in range(T):              # Collect T timesteps for 1 rollout
            action, value, log_prob = select_action(torch.FloatTensor(state)) # Get action a_t, value V(s)_t, and old policy given action a_t

            next_state, reward, terminated, truncated, _ = env.step(action.item())      # Advance simulation one time step
            done = terminated or truncated

            # Collect training data
            collect_training_data(state, action, reward, log_prob, value, done, states, actions, rewards, log_probs, values, dones)  

            state = next_state      # Move to next state
            timesteps += 1          # Increase timesteps

            if done:
                state, _ = env.reset()      # If an episode is done, reset the state
        
        # Compute advantages
        advantages = compute_advantages(next_state_tensor=torch.FloatTensor(state), rewards=rewards, values=values, dones=dones)     # Calculate advantages
        advantages = ((advantages - advantages.mean()) / (advantages.std() + 1e-8))               # Normalize advantages
        returns = advantages + torch.FloatTensor(values)        # Calculate V-target_t = A_t + V-w(s_t)

        # Convert data for training 
        states = torch.FloatTensor(states)
        actions = torch.LongTensor(actions)
        old_log_probs = torch.FloatTensor(log_probs)
        returns = returns.detach()

        # Optimizing the surrogate loss
        for k in range(K):          
            for i in torch.randperm(len(states)).split(minibatch_size):        # Sample over batches with size 'minibatch_size'      
                batch_states = states[i]
                batch_actions = actions[i]
                batch_old_log_probs = old_log_probs[i]
                batch_returns = returns[i]
                batch_advantages = advantages[i]

                optimized_value, optimized_dist, optimized_log_probs = get_parameterized_policy(batch_states=batch_states, batch_actions=batch_actions)
            
                L_clip, L_v, H_entropy = calculate_clipped_surrogate_loss(batch_old_log_probs, batch_returns, batch_advantages, optimized_value, optimized_dist, optimized_log_probs)

                # Loss = L_clip + v * L_v - h * H_entropy

                # Actor loss
                actor_loss = L_clip - h * H_entropy
                actor_losses.append(actor_loss.item())
                actor_optimizer.zero_grad()
                actor_loss.backward()
                actor_optimizer.step()

                # Critic loss
                critic_loss = L_v * v
                critic_losses.append(critic_loss.item())
                critic_optimizer.zero_grad()
                critic_loss.backward()
                critic_optimizer.step()

        if timesteps % T == 0:
            try:
                avg_rewards, std_rewards, success_rate = evaluation_and_save(eval_episodes, max_reward=max_reward)
                print(f"In current timestep: {timesteps}, Average reward: {avg_rewards}, Standard Deviation: {std_rewards}, Success rate: {success_rate}", flush=True)
                index.append(timesteps)
                eval_rewards.append(avg_rewards)
                eval_standard_deviations.append(std_rewards)
                success_rates.append(success_rate)
            except Exception as e:
                print(f"Error in evaluation: {e}", flush=True)

In [ ]:
def moving_average(data, window_size=10):
    """
    Compute the moving average of the input data.

    Parameters
    ----------
    data : array_like
        Input array or list of numerical data.
    window_size : int, optional
        Size of the moving average window, by default 10.

    Returns
    -------
    ndarray
        Array of moving averages with length len(data) - window_size + 1.
    """

    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

In [ ]:
train(env, max_timesteps, K, T, minibatch_size, max_reward, actor_losses, critic_losses, index, eval_rewards, eval_standard_deviations, success_rates)

In [ ]:
show_best_results()

In [ ]:
plt.plot(moving_average(actor_losses), c="blue")
plt.title("Actor Loss")
plt.xlabel("Rollout")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.plot(moving_average(critic_losses), c="orange")
plt.title("Critic Loss")
plt.xlabel("Rollout")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.plot(index, eval_rewards, c="red")
plt.ylim(top=300)
plt.title("Evaluation Rewards")
plt.xlabel("Timesteps")
plt.ylabel("Rewards")
plt.show()

In [ ]:
plt.plot(index, eval_standard_deviations, c="pink")
plt.title("Standard Deviation over Timesteps")
plt.xlabel("Timesteps")
plt.ylabel("STD")
plt.show()

In [ ]:
plt.plot(index, success_rates, c="green")
plt.ylim(top=100)
plt.title("Success Rate over Timesteps")
plt.xlabel("Timesteps")
plt.ylabel("Success Rates (%)")
plt.show()

# 4. Kết quả thu được

## 4.1. PPO không có gió

### 4.1.1. 1e6 Timesteps, Batchsize = 64, n_steps = 2048 (Training time: 2h50p)

| Actor Loss             | Critic Loss            | Rewards                |
| ---------------------- | ---------------------- | ---------------------- |
| ![](images/image21.png) | ![](images/image22.png) | ![](images/image23.png) |
| **Standard Deviation** | **Success Rate**       |                        |
| ![](images/image24.png) | ![](images/image25.png) |                        |

**Best reward collected**: *307.8842375695475*

<video src="best_of_ppo_1e6_64_2048_3e4\rl-video-episode-0.mp4" controls></video>

### 4.1.2. 1e6 Timesteps, Batchsize = 256, n_steps = 2048 (Training time: 2h22p)

| Actor Loss             | Critic Loss            | Rewards                |
| ---------------------- | ---------------------- | ---------------------- |
| ![](images/image26.png) | ![](images/image27.png) | ![](images/image28.png) |
| **Standard Deviation** | **Success Rate**       |                        |
| ![](images/image29.png) | ![](images/image30.png) |                        |

**Best reward collected**: *272.87406842041014*

<video src="best_of_ppo_1e6_256_2048_3e4\rl-video-episode-0.mp4" controls></video>

### 4.1.3. 1e6 Timesteps, Batchsize = 64, n_steps = 4096 (Training time: 1h20p)

| Actor Loss             | Critic Loss            | Rewards                |
| ---------------------- | ---------------------- | ---------------------- |
| ![](images/image31.png) | ![](images/image32.png) | ![](images/image33.png) |
| **Standard Deviation** | **Success Rate**       |                        |
| ![](images/image34.png) | ![](images/image35.png) |                        |

**Best reward collected**: *268.36002417594966*

<video src="best_of_ppo_1e6_64_4096_3e4\rl-video-episode-0.mp4" controls></video>

### 4.1.4. 1e6 Timesteps, Batchsize = 256, n_steps = 4096 (Training time: 1h40p)

| Actor Loss             | Critic Loss            | Rewards                |
| ---------------------- | ---------------------- | ---------------------- |
| ![](images/image36.png) | ![](images/image37.png) | ![](images/image38.png) |
| **Standard Deviation** | **Success Rate**       |                        |
| ![](images/image39.png) | ![](images/image40.png) |                        |

**Best reward collected**: *247.34049715264368*

<video src="best_of_ppo_1e6_256_4096_3e4\rl-video-episode-0.mp4" controls></video>

## 4.2. PPO có gió

### 4.2.1. 1e6 Timesteps, Batchsize = 64, n_steps = 2048 (Training time: 3h12p)

| Actor Loss             | Critic Loss            | Rewards                |
| ---------------------- | ---------------------- | ---------------------- |
| ![](images/image1.png) | ![](images/image2.png) | ![](images/image3.png) |
| **Standard Deviation** | **Success Rate**       |                        |
| ![](images/image4.png) | ![](images/image5.png) |                        |

**Best reward collected**: *273.43429085932473*

<video src="best_of_ppo_wind_1e6_64_2048_3e4\rl-video-episode-0.mp4" controls></video>

### 4.2.2. 1e6 Timesteps, Batchsize = 256, n_steps = 2048 (Training time: 3h20p)

| Actor Loss             | Critic Loss            | Rewards                |
| ---------------------- | ---------------------- | ---------------------- |
| ![](images/image6.png) | ![](images/image7.png) | ![](images/image8.png) |
| **Standard Deviation** | **Success Rate**       |                        |
| ![](images/image9.png) | ![](images/image10.png) |                        |

**Best reward collected**: *277.108768843627*

<video src="best_of_ppo_wind_1e6_256_2048_3e4\rl-video-episode-0.mp4" controls></video>

### 4.2.3. 1e6 Timesteps, Batchsize = 64, n_steps = 4096 (Training time: 2h45p)

| Actor Loss             | Critic Loss            | Rewards                |
| ---------------------- | ---------------------- | ---------------------- |
| ![](images/image11.png) | ![](images/image12.png) | ![](images/image13.png) |
| **Standard Deviation** | **Success Rate**       |                        |
| ![](images/image14.png) | ![](images/image15.png) |                        |

**Best reward collected**: *286.28957209302223*

<video src="best_of_ppo_wind_1e6_64_4096_3e4\rl-video-episode-0.mp4" controls></video>

### 4.2.4. 1e6 Timesteps, Batchsize = 256, n_steps = 4096 (Training time: 2h40p)

| Actor Loss             | Critic Loss            | Rewards                |
| ---------------------- | ---------------------- | ---------------------- |
| ![](images/image16.png) | ![](images/image17.png) | ![](images/image18.png) |
| **Standard Deviation** | **Success Rate**       |                        |
| ![](images/image19.png) | ![](images/image20.png) |                        |

**Best reward collected**: *271.1365183294953*

<video src="best_of_ppo_wind_1e6_256_4096_3e4\rl-video-episode-0.mp4" controls></video>

# 5. Tổng kết quá trình huấn luyện
Qua kết quả thu được, em nhận thấy rằng việc sử dụng thuật toán Proximal Policy Optimization (PPO) mang lại hiệu quả huấn luyện cao hơn rõ rệt so với Advantage Actor Critic (A2C). Nguyên nhân chính đến từ tính ổn định cao của PPO: thuật toán này giới hạn độ thay đổi giữa các chính sách cũ và mới thông qua hàm phạt (clipping), từ đó tránh được tình trạng cập nhật quá lớn làm hỏng chính sách đang học. Nhờ vậy, quá trình huấn luyện diễn ra trơn tru hơn, ít dao động mạnh, và giúp Agent dễ dàng hội tụ đến chiến lược tối ưu.

Bên cạnh đó, việc lựa chọn và tinh chỉnh các siêu tham số như learning rate, batch size, discount factor, v.v., cũng đóng vai trò rất quan trọng. Với bộ siêu tham số phù hợp, thuật toán PPO đã nhanh chóng đạt được sự hội tụ: Agent không những học được cách chơi mà còn thể hiện được những hành vi thông minh, chủ động đưa ra chiến lược tối ưu trong môi trường.

Ngoài ra, qua quá trình thực nghiệm, em cũng nhận thấy PPO có khả năng khái quát tốt hơn, tức là Agent không chỉ học thuộc một vài tình huống cụ thể mà có thể thích ứng linh hoạt với các trạng thái khác nhau trong môi trường (không có gió, có gió...). Điều này cho thấy tiềm năng lớn của PPO khi áp dụng vào các bài toán phức tạp hơn trong thực tế.

Tuy nhiên, vẫn còn một số hạn chế cần lưu ý, chẳng hạn như thời gian huấn luyện khá lâu do PPO yêu cầu lượng sample lớn hơn để đạt được sự ổn định. Trong tương lai, có thể xem xét các cải tiến như sử dụng kỹ thuật Early Stopping, điều chỉnh siêu tham số, tối ưu thời gian chạy, ....

Nếu đã đọc đến đây, em xin trân trọng cảm ơn thầy vì đã xem qua bài báo cáo trình bày của em. Vì đây là lần đầu tiên huấn luyện mô hình nên trong quá trình viết báo cáo và thực hành huấn luyện có thể có thiếu sót, hoặc có đôi chỗ em chưa nắm vững kiến thức, mong thầy thông cảm và bỏ qua.

Chúc thầy một ngày tốt lành ! 

# 6. Tài liệu tham khảo
1. Thông tin về môi trường huấn luyện Lunar Lander: [Lunar Lander Gymnasium Documentation](https://gymnasium.farama.org/environments/box2d/lunar_lander/)
2. Kiến thức về Advantage Actor Critic (A2C) cho người mới: [Actor Critic methods with Robotics environments](https://huggingface.co/learn/deep-rl-course/unit6/introduction)
3. Kiến thức về Proximal Policy Optimization (PPO) cho người mới: [Proximal Policy Optimization (PPO)](https://huggingface.co/learn/deep-rl-course/unit6/introduction)
4. Kiến thức về Advantage Actor Critic (A2C) trên trang DataCamp: [Advantage Actor Critic](https://campus.datacamp.com/courses/deep-reinforcement-learning-in-python/introduction-to-policy-gradient-methods?ex=7)
5. Kiến thức về Proximal Policy Optimization (PPO) trên trang DataCamp: [Proximal Policy Optimization](https://projector-video-pdf-converter.datacamp.com/36398/chapter4.pdf)
6. Paper chính thức của A2C: [Asynchronous Methods for Deep Reinforcement Learning](https://arxiv.org/pdf/1602.01783) 
7. Paper chính thức của PPO: [Proximal Policy Optimization Algorithms](https://arxiv.org/pdf/1707.06347)
8. Paper chính thức của GAE: [High-Dimensional Continuous Control Using Generalized Advantage Estimation](https://arxiv.org/pdf/1506.02438)
9. Dự án nghiên cứu Thạc sĩ: [Towards Delivering a Coherent Self-Contained Explanation of Proximal Policy Optimization](https://fse.studenttheses.ub.rug.nl/25709/1/mAI_2021_BickD.pdf)

In [6]:
import base64
import json
import os
import re

# Paths to media
image_dir = './images/'
input_notebook = 'PPO_LunarLander.ipynb'
output_notebook = 'PPO_LunarLander_Embedded5.ipynb'

# Function to encode file to base64
def file_to_base64(filepath):
    with open(filepath, 'rb') as file:
        return base64.b64encode(file.read()).decode('utf-8')

# Collect all image files
image_files = [f'image{i}.png' for i in range(1, 52)]  # image1.png to image40.png

# Load the notebook
with open(input_notebook, 'r', encoding='utf-8') as f:
    notebook = json.load(f)

# Process each cell
for cell in notebook['cells']:
    if cell['cell_type'] == 'markdown':
        source = ''.join(cell['source'])
        # Replace image references
        for img in image_files:
            img_path = os.path.join(image_dir, img)
            if os.path.exists(img_path):
                img_b64 = file_to_base64(img_path)
                img_pattern = rf'!\[\]\(images/{img}\)'
                img_data_uri = f'![{img}](data:image/png;base64,{img_b64})'
                source = re.sub(img_pattern, img_data_uri, source)
        # Update cell source
        cell['source'] = source.splitlines(keepends=True)

# Save the new notebook
with open(output_notebook, 'w', encoding='utf-8') as f:
    json.dump(notebook, f, indent=2)

print(f"Embedded notebook saved as {output_notebook}")

Embedded notebook saved as PPO_LunarLander_Embedded5.ipynb
